# Multi-Tenant Data Isolation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/multi_tenant_isolation/multi_tenant_isolation.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/multi_tenant_isolation/multi_tenant_isolation.ipynb)

---

## 🏢 Business Scenario

You operate a **B2B SaaS analytics platform** serving 50+ enterprise clients.
Each tenant's clickstream data lives in a shared ADLS account, isolated by tenant namespace:

```
abfss://events@myaccount.dfs.core.windows.net/
  prod/
    acme-corp/    bronze/  silver/  gold/
    globex-ltd/   bronze/  silver/  gold/
    initech/      bronze/  silver/  gold/
```

Without a promotion strategy:
- 50 tenants × 3 layers × 3 environments = **450 YAML files** to maintain
- Schema drift between tenants is invisible until prod failures
- Onboarding a new tenant requires copying and editing 9 YAMLs

## 💡 Value Proposition

- ✅ **3 contract templates** — `bronze_events.yaml`, `silver_events.yaml`, `gold_tenant_kpi.yaml`
- ✅ **`LAKELOGIC_ENV=dev-acme`** routes to ACME dev; `prod-globex` routes to Globex prod
- ✅ **Same schema, quality rules, SLAs** enforced across all tenants
- ✅ **Onboarding = add 3 environment entries** + seed data; no new YAML files
- ✅ **Zero cross-tenant leakage** — path isolation enforced by the contract model

---

## Goals

1. Inspect per-tenant `environments` blocks
2. Run Bronze → Silver → Gold for **ACME Corp** (`dev-acme`)
3. Run the same contracts for **Globex Ltd** (`dev-globex`)
4. Compare the KPI outputs side-by-side
5. See the onboarding pattern for a new tenant


---
## Step 0 — Setup


In [ ]:
import os
import yaml
from pathlib import Path

import polars as pl

from lakelogic import DataProcessor
from lakelogic.core.models import DataContract

def resolve_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        if base.name == name and base.exists():
            return base
        for candidate in [
            base / name,
            base / "examples" / "06_advanced_workflows" / name,
            base / "lakelogic" / "examples" / "06_advanced_workflows" / name,
        ]:
            if candidate.exists():
                return candidate
    return cwd / name

EXAMPLE_DIR = resolve_example_dir("multi_tenant_isolation")
DATA_DIR    = EXAMPLE_DIR / "data"
CONTRACTS   = EXAMPLE_DIR / "contracts"

TENANTS = ["acme", "globex"]

# Clean previous dev outputs
for tenant in TENANTS:
    for f in (DATA_DIR / "dev" / tenant).glob("*.parquet"):
        f.unlink()

print(f"Example dir: {EXAMPLE_DIR}")
print(f"Tenant data:")
for tenant in TENANTS:
    csv = DATA_DIR / "dev" / tenant / "events_raw.csv"
    print(f"  {tenant}: {csv} ({'exists' if csv.exists() else 'MISSING'})")


---
## Step 1 — Inspect per-tenant environments

The `environments` block uses `{env}-{tenant}` keys.
Same contracts, different paths.


In [ ]:
bronze_contract = DataContract(**yaml.safe_load((CONTRACTS / "bronze_events.yaml").read_text()))

print("bronze_events.yaml — environments block:")
print()
print(f"  {'Key':<20}  {'Path':<65}  {'Format'}")
print(f"  {'---':<20}  {'----':<65}  {'------'}")
print(f"  {'(default/prod)':<20}  {bronze_contract.server.path:<65}  {bronze_contract.server.format}")
for env_name, env_cfg in bronze_contract.environments.items():
    fmt = env_cfg.format or bronze_contract.server.format
    print(f"  {env_name:<20}  {env_cfg.path:<65}  {fmt}")


---
## Step 2 — Run pipeline for ACME Corp (`dev-acme`)


In [ ]:
os.environ["LAKELOGIC_ENV"] = "dev-acme"
print(f"Active env: {os.environ['LAKELOGIC_ENV']}")

acme_src = DATA_DIR / "dev" / "acme" / "events_raw.csv"
acme_bronze_out = DATA_DIR / "dev" / "acme" / "bronze_events.parquet"
acme_silver_out = DATA_DIR / "dev" / "acme" / "silver_events.parquet"
acme_gold_out   = DATA_DIR / "dev" / "acme" / "gold_tenant_kpi.parquet"


In [ ]:
# Bronze
bronze_contract = DataContract(**yaml.safe_load((CONTRACTS / "bronze_events.yaml").read_text()))
bronze_contract._base_path = CONTRACTS

print(f"ACME bronze source → {bronze_contract.effective_server().path}")

result = DataProcessor(engine="polars", contract=bronze_contract).run_source(str(acme_src))
result.good.write_parquet(acme_bronze_out)
print(f"✅ Bronze: {result.good.height} good rows  ❌ {result.bad.height if result.bad is not None else 0} bad rows")


In [ ]:
# Silver
silver_contract = DataContract(**yaml.safe_load((CONTRACTS / "silver_events.yaml").read_text()))
silver_contract._base_path = CONTRACTS

result_s = DataProcessor(engine="polars", contract=silver_contract).run_source(str(acme_bronze_out))
result_s.good.write_parquet(acme_silver_out)
print(f"✅ Silver: {result_s.good.height} good rows")
result_s.good.select(["event_id","tenant_id","event_type","event_hour"]).head(5)


In [ ]:
# Gold
gold_contract = DataContract(**yaml.safe_load((CONTRACTS / "gold_tenant_kpi.yaml").read_text()))
gold_contract._base_path = CONTRACTS

result_g = DataProcessor(engine="polars", contract=gold_contract).run_source(str(acme_silver_out))
result_g.good.write_parquet(acme_gold_out)

print(f"ACME Gold KPI ({result_g.good.height} rows):")
result_g.good.sort(["event_date","event_type"])


---
## Step 3 — Run the same contracts for Globex Ltd (`dev-globex`)

No contract changes — just flip the env var.


In [ ]:
os.environ["LAKELOGIC_ENV"] = "dev-globex"
print(f"Active env: {os.environ['LAKELOGIC_ENV']}")

globex_src        = DATA_DIR / "dev" / "globex" / "events_raw.csv"
globex_bronze_out = DATA_DIR / "dev" / "globex" / "bronze_events.parquet"
globex_silver_out = DATA_DIR / "dev" / "globex" / "silver_events.parquet"
globex_gold_out   = DATA_DIR / "dev" / "globex" / "gold_tenant_kpi.parquet"


In [ ]:
# Full pipeline in one cell for Globex
for contract_file, src, out in [
    ("bronze_events.yaml", globex_src, globex_bronze_out),
    ("silver_events.yaml", globex_bronze_out, globex_silver_out),
    ("gold_tenant_kpi.yaml", globex_silver_out, globex_gold_out),
]:
    c = DataContract(**yaml.safe_load((CONTRACTS / contract_file).read_text()))
    c._base_path = CONTRACTS
    r = DataProcessor(engine="polars", contract=c).run_source(str(src))
    r.good.write_parquet(out)
    print(f"✅ {contract_file.replace('.yaml',''):<25} → {out.name}  ({r.good.height} rows)")

print()
print("Globex Gold KPI:")
globex_gold = pl.read_parquet(globex_gold_out)
globex_gold.sort(["event_date","event_type"])


---
## Step 4 — Compare KPIs across tenants

Each tenant's Gold output is fully isolated — running the pipeline for ACME
can never write to Globex's namespace and vice versa.


In [ ]:
acme_gold   = pl.read_parquet(acme_gold_out).with_columns(pl.lit("acme").alias("tenant"))
globex_gold = pl.read_parquet(globex_gold_out).with_columns(pl.lit("globex").alias("tenant"))

combined = pl.concat([acme_gold, globex_gold])

summary = (
    combined
    .group_by("tenant")
    .agg([
        pl.col("event_count").sum().alias("total_events"),
        pl.col("unique_users").sum().alias("total_unique_users"),
        pl.col("unique_sessions").sum().alias("total_sessions"),
    ])
    .sort("tenant")
)

print("Cross-tenant KPI comparison:")
print(summary)
print()
print("Note: data paths are isolated — this comparison is done by reading both Gold outputs.")
print("In production, tenants can only query their own Gold namespace.")


---
## Step 5 — Onboarding a new tenant (Initech)

Three steps:
1. Add `dev-initech` and `prod-initech` entries to each contract's `environments` block
2. Drop the tenant's seed CSV in `data/dev/initech/events_raw.csv`
3. Run: `LAKELOGIC_ENV=dev-initech lakelogic-driver ...`

No schema changes, no quality rule duplication, no new YAML files.


In [ ]:
# Simulate adding a new tenant's environments entry programmatically
import copy

bronze_data = yaml.safe_load((CONTRACTS / "bronze_events.yaml").read_text())

# What a new onboarding team member would add
new_tenant_entries = {
    "dev-initech":  {"path": "./data/dev/initech/events_raw.csv", "format": "csv"},
    "test-initech": {"path": "abfss://events@myaccount.dfs.core.windows.net/test/initech/bronze/"},
    "prod-initech": {"path": "abfss://events@myaccount.dfs.core.windows.net/prod/initech/bronze/"},
}

print("Environments that would be added to bronze_events.yaml for Initech:")
print(yaml.dump(new_tenant_entries, default_flow_style=False))
print("Then run: LAKELOGIC_ENV=dev-initech lakelogic-driver --registry contracts/_registry.yaml --layers bronze,silver,gold")


---
## Summary

| Tenant | Env | LAKELOGIC_ENV | Source path |
|--------|-----|---------------|-------------|
| ACME   | dev | `dev-acme`    | `./data/dev/acme/events_raw.csv` |
| ACME   | prod| (none)→ override | `abfss://.../prod/acme/bronze/` |
| Globex | dev | `dev-globex`  | `./data/dev/globex/events_raw.csv` |
| Globex | prod| (none)→ override | `abfss://.../prod/globex/bronze/` |

**3 contract files** serve all tenants across all environments.
Adding a 51st tenant takes **3 minutes** — add 3 environment entries, drop a CSV, run.

### Next steps
- `08_compliance_governance/hipaa_pii_masking/` — per-tenant PII masking with contractual guarantees
- `06_advanced_workflows/gdpr_compliance/` — tenant-scoped GDPR erasure
- `07_production/ci_cd/` — multi-tenant parallel CI/CD orchestration
